## **Распределительная задача**

Завод может производить Y = 25 видов продукции. На продукцию есть покупатель, которому требуется определенное количество каждого вида:

Таблица M (кг): M = [52 37 65 64 66 66 62 49 33 54 49 67 60 68 69 35 45 53 62 40 32 54 64 60 47]

Для каждого вида продукции он либо покупает требуемое количество, либо не покупает совсем, если мы произвели недостаточно. При этом известна цена контракта на каждый вид продукции:

Таблица C (рубли): C = [656 672 675 665 512 620 451 538 523 670 591 421 574 531 608 454 471 414 605 543 612 514 638 444 437]

Завод одновременно может выпускать только один вид продукции со скоростью:

Таблица P (кг в сутки): P = [19 15 27 19 29 16 22 18 16 27 21 21 24 25 28 21 24 21 15 15 17 29 27 24 21]

Требуется написать математическую модель и найти оптимальный план производства на S = 20 суток с помощью Gurobi, если известно, что переход от одной продукции к другой происходит мгновенно, а его стоимость R = 49 рублей.


#### Запишем математическую модель:

Введём переменные:

$$x_{i} = \begin{cases}
   1,&\text{если заключаем контракт с } i &\text{заводом,}\\
   0,&\text{в противном случае.}
\end{cases}$$

Целевая функция:

$$\sum\limits_{i = 0}^{Y} (c_i - R)x_i + R\to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i \in Y $$

$$\sum\limits_{i = 0}^{Y} x_i \lceil m_i/p_i \rceil \leqslant S $$

In [14]:
import gurobipy as gb
from gurobipy import GRB
import numpy as np

In [15]:
Y = 25
S = 20
R = 49

M = np.array([52, 37, 65, 64, 66, 66, 62, 49, 33, 54, 49, 67, 60, 68, 69, 35, 45, 53, 62, 40, 32, 54, 64, 60, 47])
C = np.array([656, 672, 675, 665, 512, 620, 451, 538, 523, 670, 591, 421, 574, 531, 608, 454, 471, 414, 605, 543, 612, 514, 638, 444, 437])
P = np.array([19, 15, 27, 19, 29, 16, 22, 18, 16, 27, 21, 21, 24, 25, 28, 21, 24, 21, 15, 15, 17, 29, 27, 24, 21])

In [16]:
problem = gb.Model('Distribution_problem')

In [17]:
factory = problem.addVars(range(0, Y), vtype=gb.GRB.BINARY, name='factory')

obj = gb.LinExpr(C - R, factory.values()) + R
problem.setObjective(obj, gb.GRB.MAXIMIZE)
problem.addConstr(gb.LinExpr(np.ceil(M / P), factory.values()) <= S)

problem.update()


In [18]:
problem.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 25 columns and 25 nonzeros
Model fingerprint: 0x32e5df54
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [2e+00, 5e+00]
  Objective range  [4e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 3386.0000000
Presolve removed 1 rows and 25 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 4565 3386 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.565000000000e+03, best bound 4.565000000000e+03, gap 0.0000%


In [45]:
print(f'Заключили контракт c заводами: {[var.index + 1 for var in problem.getVars() if var.X]}')
print(f'Максимальная прибыль: {problem.objVal}')

Заключили контракт c заводами: [1, 2, 3, 10, 17, 21, 22, 23]
Максимальная прибыль: 4565.0
